In [52]:
import json
import os
from typing import Optional, List

from dotenv import load_dotenv
from geojson import FeatureCollection
from mapboxgl.utils import create_color_stops
from mapboxgl.viz import LinestringViz
from ortools.constraint_solver.routing_enums_pb2 import LocalSearchMetaheuristic, FirstSolutionStrategy

from src.geometry.geometry import shorten_line
from src.helpers.helpers import format_matrix
from src.map.map import satellite_style
from src.optimiser.connections_optimiser import ConnectionsOptimiser
from src.optimiser.lanelets_optimiser import LaneletsOptimiser
from src.osrm.interface import OSRMInterface
from src.routing_problem.connections.complete import CompleteConnection, LastConnection
from src.routing_problem.connections.lanelet import LaneletConnection
from src.routing_problem.connections.route import RouteConnection
from src.routing_problem.creator.creator import create_routing_problem
from src.routing_problem.lanelet import Lanelet
from src.routing_problem.routing_problem import RoutingProblem

In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
load_dotenv()
token = os.getenv('MAPBOX_ACCESS_TOKEN')
rp_path = "data/stuttgart_small.osm"

In [55]:
routing_problem: RoutingProblem = create_routing_problem(rp_path)
geojson = FeatureCollection(features=routing_problem.to_json())

viz = LinestringViz(data=geojson,
                    color_default="#3ad21b",
                    line_width_default='2',
                    center=(9.092, 48.731),
                    zoom=18,
                    style=satellite_style)
viz.show()

In [56]:
from src.routing_problem.maneuver.maneuver_type import ManeuverType
from src.routing_problem.maneuver.modifier import ManeuverModifier

# Load routing problem
routing_problem: RoutingProblem = create_routing_problem(rp_path)

# Shorten lanelets
for lanelet in routing_problem.lanelets:
    if lanelet.get_length() > 50:
        lanelet.nodes = shorten_line(lanelet.nodes, 12, cut_beginning=True)
        lanelet.nodes = shorten_line(lanelet.nodes, 20, cut_beginning=False)
    elif lanelet.get_length() > 30:
        lanelet.nodes = shorten_line(lanelet.nodes, 5, cut_beginning=True)
        lanelet.nodes = shorten_line(lanelet.nodes, 5, cut_beginning=False)
    elif lanelet.get_length() > 10:
        lanelet.nodes = shorten_line(lanelet.nodes, 3, cut_beginning=True)
        lanelet.nodes = shorten_line(lanelet.nodes, 3, cut_beginning=False)
    elif lanelet.get_length() > 5:
        lanelet.nodes = shorten_line(lanelet.nodes, 1, cut_beginning=True)
        lanelet.nodes = shorten_line(lanelet.nodes, 1, cut_beginning=False)

maneuvers = dict()
for segment in routing_problem.segments:
    maneuvers.update(segment.next_maneuvers)

# Create connections. First for left and right turns, then for forward.
connections = []
for segment in routing_problem.segments:
    for previous_segment in segment.previous_segments:
        maneuver = maneuvers[(previous_segment.id, segment.id)]
        left = maneuver.modifier in [ManeuverModifier.Left, ManeuverModifier.SlightLeft,
                                     ManeuverModifier.SharpLeft, ManeuverModifier.UTurn]
        right = maneuver.modifier in [ManeuverModifier.Right, ManeuverModifier.SlightRight,
                                      ManeuverModifier.SharpRight]
        merge = maneuver.type == ManeuverType.Merge
        # Left
        if left and not merge or right and merge:
            for i, lanelet in enumerate(segment.lanelets):
                if i < len(previous_segment.lanelets):
                    connections.append(LaneletConnection(previous_segment.lanelets[i],
                                                         lanelet,
                                                         maneuver))
        # Right
        elif right and not merge or left and merge:
            for i, lanelet in enumerate(reversed(segment.lanelets)):
                if i < len(previous_segment.lanelets):
                    connections.append(LaneletConnection(previous_segment.lanelets[-(i + 1)],
                                                         lanelet,
                                                         maneuver))

# Now connect straights
for segment in routing_problem.segments:
    for previous_segment in segment.previous_segments:
        maneuver = maneuvers[(previous_segment.id, segment.id)]
        if maneuver.modifier != ManeuverModifier.Straight:
            continue

        lanelets_to = len(segment.lanelets)
        lanelets_from = len(previous_segment.lanelets)
        free_lanelets_to = list(filter(lambda x: not x.has_incoming_connection, segment.lanelets))
        free_lanelets_from = list(filter(lambda x: not x.has_outgoing_connection, previous_segment.lanelets))
        free_lanelets_to_count = len(free_lanelets_to)
        free_lanelets_from_count = len(free_lanelets_from)

        if lanelets_to == lanelets_from:
            for i in range(lanelets_to):
                connections.append(LaneletConnection(previous_segment.lanelets[i],
                                                     segment.lanelets[i],
                                                     maneuver))
        elif free_lanelets_to_count == lanelets_from:
            for i in range(free_lanelets_to_count):
                connections.append(LaneletConnection(previous_segment.lanelets[i],
                                                     free_lanelets_to[i],
                                                     maneuver))
        elif lanelets_to == free_lanelets_from_count:
            for i in range(lanelets_to):
                connections.append(LaneletConnection(free_lanelets_from[i],
                                                     segment.lanelets[i],
                                                     maneuver))
        elif lanelets_to > free_lanelets_from_count:
            for i in range(lanelets_to):
                connections.append(LaneletConnection(free_lanelets_from[min(i, free_lanelets_from_count - 1)],
                                                     segment.lanelets[i],
                                                     maneuver))
        elif lanelets_to < free_lanelets_from_count:
            for i in range(free_lanelets_from_count):
                connections.append(LaneletConnection(free_lanelets_from[i],
                                                     segment.lanelets[min(i, lanelets_to - 1)],
                                                     maneuver))

# Convert stuff to geojson
rp_json = routing_problem.to_json()
connections_json = [connection.to_json() for connection in connections]
geojson = FeatureCollection(features=rp_json + connections_json)

# Show everything on map
color_breaks = [0, 1]
color_stops = create_color_stops(color_breaks, colors=["#3ad21b", "#ff0505"])
viz = LinestringViz(data=geojson,
                    color_property="type",
                    color_stops=color_stops,
                    line_width_default='2',
                    center=(9.092, 48.731),
                    zoom=18,
                    style=satellite_style)
viz.show()

In [57]:
sources = [int(segment.id) for segment in routing_problem.segments]
destinations = sources

try:
    durations_matrix = OSRMInterface.request_table(sources, destinations, "Stuttgart/stuttgart-regbez-latest.osrm")[1]
except:
    pass
else:
    matrix = format_matrix(sources, destinations, durations_matrix)

    with open("data/matrix.json", "w") as f:
        json.dump(matrix, f)
    print(f"Dumped {len(matrix)} x {len(matrix)} durations matrix")

In [58]:
with open("data/matrix.json") as f:
    matrix = json.load(f)

In [59]:
lanelet_connections = {(connection.lanelet_from, connection.lanelet_to) for connection in connections}
optimiser = LaneletsOptimiser(lanelets=routing_problem.lanelets,
                              matrix=matrix,
                              local_search_metaheuristic=LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
                              first_solution_strategy=FirstSolutionStrategy.AUTOMATIC,
                              max_optimisation_duration=2,
                              connections=lanelet_connections)
optimal_order, optimization_history = optimiser.optimise()
print(f"Optimisation history: {optimization_history}")

Optimisation history: {'0.0': 6169, '0.5': 5445, '1.0': 5444, '1.5': 5444}


In [60]:
route_connections = []

previous_connection: Optional[Lanelet] = None
for lanelet in optimal_order:
    if previous_connection is not None:
        cost = previous_connection.get_cost_to(lanelet,
                                            matrix,
                                            lanelet_connections)
        # if cost < 20:
        route_connections.append(RouteConnection(previous_connection,
                                                 lanelet,
                                                 cost,
                                                 1))
    previous_connection = lanelet

# Convert stuff to geojson
rp_json = routing_problem.to_json()
connections_json = [connection.to_json() for connection in route_connections]
geojson = FeatureCollection(features=rp_json + connections_json)

# Show everything on map
color_breaks = [0, 1]
color_stops = create_color_stops(color_breaks, colors=["#3ad21b", "#ff0505"])
viz = LinestringViz(data=geojson,
                    color_property="type",
                    color_stops=color_stops,
                    line_width_default='2',
                    center=(9.092, 48.731),
                    zoom=18,
                    style=satellite_style)
viz.show()

In [61]:
from src.routing_problem.connections.complete import FirstConnection

complete_connections: List[CompleteConnection] = [FirstConnection()]
disjunctions: List[List[int]] = []

for lanelet_to in routing_problem.lanelets:
    disjunctions.append([])
    segment_to = lanelet_to.segment

    for lanelet_from in routing_problem.lanelets:
        segment_from = lanelet_from.segment

        if (segment_from.id, segment_to.id) in maneuvers or len(segment_to.previous_segments) == 0:
            maneuver = maneuvers[(segment_from.id, segment_to.id)] if (segment_from.id,
                                                                       segment_to.id) in maneuvers else None

            complete_connections.append(CompleteConnection(lanelet_from=lanelet_from,
                                                           lanelet_to=lanelet_to,
                                                           maneuver=maneuver))
            disjunctions[-1].append(len(complete_connections) - 1)

complete_connections.append(LastConnection())
print(f"Created {len(complete_connections)} complete connections")

Created 627 complete connections


In [62]:
nodes_count = 0
for lanelet in routing_problem.lanelets:
    for node in lanelet.segment.nodes:
        nodes_count += 1

print(f"Segments: {len(routing_problem.segments)}")
print(f"Node based: {nodes_count}")
print(f"Edge based: {len(routing_problem.lanelets)}")
print(f"X-Graph: {len(complete_connections)}")

Segments: 33
Node based: 549
Edge based: 60
X-Graph: 627


In [63]:
optimiser = ConnectionsOptimiser(connections=complete_connections,
                                 disjunctions=disjunctions,
                                 matrix=matrix,
                                 local_search_metaheuristic=LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
                                 first_solution_strategy=FirstSolutionStrategy.AUTOMATIC,
                                 max_optimisation_duration=600)
optimal_order, optimization_history = optimiser.optimise()
print(f"Optimisation history: {optimization_history}")
optimal_order = optimal_order[1:-1]

Optimisation history: {'0.9': 4480, '1.7': 4479, '2.3': 2948, '2.9': 2922, '3.6': 2922, '4.2': 2922, '4.7': 2922, '5.2': 2922, '5.7': 2922, '6.3': 2922, '6.8': 2640, '7.5': 2618, '8.0': 2618, '8.5': 2618, '9.0': 2618, '9.5': 2617, '10.1': 2617, '10.6': 2617, '11.1': 2617, '11.8': 2617, '12.4': 2617, '12.9': 2617, '13.5': 2617, '14.2': 2617, '14.7': 2617, '15.2': 2617, '15.8': 2617, '16.3': 2617, '16.9': 2351, '17.6': 2320, '18.1': 2265, '18.6': 2265, '19.2': 2265, '19.8': 2265, '20.3': 2265, '20.9': 2265, '21.4': 2265, '22.0': 2265, '22.5': 2265, '23.0': 2265, '23.6': 2265, '24.1': 2265, '24.6': 2265, '25.1': 2265, '25.7': 2265, '26.3': 2265, '26.8': 2265, '27.4': 2265, '27.9': 2265, '28.5': 2265, '29.2': 2265, '29.7': 2265, '30.3': 2265, '30.8': 2265, '31.4': 2265, '32.0': 2265, '32.5': 2265, '33.1': 2265, '33.7': 2265, '34.3': 2265, '34.8': 2265, '35.3': 2265, '35.8': 2265, '36.4': 2265, '36.9': 2265, '37.4': 2265, '37.9': 2265, '38.6': 2265, '39.1': 2265, '39.8': 2265, '40.3': 2265,

In [64]:
route_connections = []

previous_connection: Optional[CompleteConnection] = None
for connection in optimal_order:
    if previous_connection is not None:
        cost = previous_connection.get_cost_to(connection,
                                               matrix)
        # route_connections.append(RouteConnection(previous_connection.lanelet_to,
        #                                          connection.lanelet_from,
        #                                          cost,
        #                                          1))
        route_connections.append(RouteConnection(previous_connection.lanelet_from,
                                                 previous_connection.lanelet_to,
                                                 0,
                                                 2))
    previous_connection = connection

# Convert stuff to geojson
rp_json = routing_problem.to_json()
connections_json = [connection.to_json() for connection in route_connections]
geojson = FeatureCollection(features=rp_json + connections_json)

# Show everything on map
color_breaks = [0, 1, 2]
color_stops = create_color_stops(color_breaks, colors=["#3ad21b", "#ff0505", "#d742f5"])
viz = LinestringViz(data=geojson,
                    color_property="type",
                    color_stops=color_stops,
                    line_width_default='2',
                    center=(9.092, 48.731),
                    zoom=18,
                    style=satellite_style)
viz.show()